# Group Project
### Members:
- Minh Au
- Kevin
- Veronika
- Andrew

## Preliminary exploratory data analysis
To give brief information about the dataset we chose, the following section involves the preliminiary data analysis. First and foremost, necessary libraries need to be loaded

In [5]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

Subseqeuently, the data can be read into a dataframe. Note that the dataset was downloaded from kaggle and uploaded to Jupyter notebook beforehand.
Source: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset?resource=download.

In [11]:
# Reading the ata
heart_data <- read_csv("data/heart.csv")
head(heart_data, 5)

Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


As seen in the first five rows of the data, we could see that the dataset is already in the tidy format. This can be seen with how every column is a variable, every row is a single observation and every cell only has one value. Although, we still need modify the dataset slightly to suit the classification problem. More specifically, since we are trying to predict the type of chest pain, it needs to be represented as a category/factor instead of a number. Similarly, other categorical attributes include: sex, fasting blood sugar larger than $120$ mg/dl, resting electrocardiographic results, exercise induced agina, number of major vessels, and thalassemia.
# age, sex, chol, smoking (cigarettes per day or number of years as smoker), max heart rate, rest heart rate, resting blood pressure

In [1]:
# Convert some attributes to factors
heart_data <- heart_data |>
    mutate(sex = as_factor(sex), cp = as_factor(cp), fbs = as_factor(fbs), restecg = as_factor(restecg),
           exang = as_factor(exang), ca = as_factor(ca), thal = as_factor(thal))
head(heart_data, 5)

ERROR: Error in mutate(heart_data, sex = as_factor(sex), cp = as_factor(cp), : could not find function "mutate"


Before processing the data any further, we must extract the training data to prevent the test data from affecting any aspect of our data analysis process. As there are 1025 rows (or 1025 different observations), an appropriate proportion between training and testing dataset is $80\%$. Furthermore, to prevent any bias, the splitting will be random while preserving the good proportion of different chest pain type in both data subset. Note that for reproducibility, random seed of $1337$ will be used.

In [23]:
# Splitting the data into training and testing
set.seed(1337)
heart_split <- initial_split(heart_data, prop = 0.8, strata = cp)
heart_train <- training(heart_split)
heart_test <- testing (heart_split)

In [43]:
# nrow(heart_train)
# heart_train_missing <- heart_train |>
#     mutate(cp = mean(cp))

# heart_train_missing

[1] 820

Warning message in mean.default(cp):
“argument is not numeric or logical: returning NA”


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>
52,1,NA,125,212,0,1,168,0,1.0,2,2,3,0
70,1,NA,145,174,0,1,125,1,2.6,0,0,3,0
62,0,NA,138,294,1,1,106,0,1.9,1,3,2,0
58,0,NA,100,248,0,0,122,0,1.0,1,0,2,1
58,1,NA,114,318,0,2,140,0,4.4,0,3,1,0
55,1,NA,160,289,0,0,145,1,0.8,1,1,3,0
46,1,NA,120,249,0,0,144,0,0.8,2,0,3,0
71,0,NA,112,149,0,1,125,0,1.6,1,0,2,1
51,1,NA,140,298,0,1,122,1,4.2,1,3,3,0
